# Numerical Integration
Computing a numerical integral (or quadrature <https://en.wikipedia.org/wiki/Numerical_integration?useskin=vector>) is a typical problem that appears everywhere in numerical computing. The simple task of computing the error function, $f(x) = \int_{0}^x e^{-t^2}$, requires the application of numerical techniques to calculate this (improper integral) over an arbitrary range.  

In any numerical computing course you will start by exploring the trapezoidal and the simpsons rule. Here we are jus going to give the general formulation and then go directly to the functions given by scipy

Let's compare several of the typical algorithms: <https://www.geogebra.org/m/WsZgaJcc>, and a beautiful visualization at <https://www.youtube.com/watch?v=FnJqaIESC2s>

All numerical quadratures can be written as 

\begin{equation}
\int_a^b dx\  f(x) \simeq \sum_i w_i f(x_i) \Delta x + \epsilon,
\end{equation}
where the weights $w_i$ and the error $\epsilon$ depend on the method chosen (see <https://en.wikipedia.org/wiki/Newton%E2%80%93Cotes_formulas?useskin=vector>).

## Trapezoidal rule
Here the function between two points is approximated by a straight line and the te area covered by the trapezoid is added up. See: <https://en.wikipedia.org/wiki/Trapezoidal_rule>

|<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Integration_num_trapezes_notation.svg/330px-Integration_num_trapezes_notation.svg.png" /> |<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Trapezium2.gif/330px-Trapezium2.gif" />|
|-|-|
|1-order interpolation|Better results for more intervals|


Between two points, the integral is approximated as 

\begin{equation}
I (a, b) = \int_a^b dx \ f(x) \simeq \frac{b-a}{2} (f(a) + f(b)) + O_{local}(\Delta x^3),
\end{equation}
where the weights are ${1/2, 1/2}$.

The integral between two points, $a$ and $b$, after dividing its length in $N$ intervals (N+1 points, regular partition) of size $\Delta x = \dfrac{b-a}{N}$, is given by

$$
\int_a^b f(x) dx \simeq \Delta x \left( \frac{f(x_0)}{2} + \sum_{k=1}^{N-1} f(x_k) +\frac{f(x_N)}{2}\right) + O_{global}(\Delta x^{2}),
$$

where $x_k = x_0 + k \Delta x$, $x_0 = a$, and $x_N=b$. The local error is accumulated $N$ times, so $O_{global} = N O_{local} = N*(b-a)/N^3 = (b-a)/N^2 = O_{global}(N^{-2}) = O_{global}(\Delta x^{2}) $.

:::{exercise}

Now, please create a function to compute the integral of a function ($x^2$) using the trapezoid algorithm. 

OPTIONAL: Write the declaration and implementation on in `integration.h` and `integration.cpp`, respectively. 

It must fulfill the following tests

```c++
std::cout << trapezoid(0.0, 1.0, 10, fun) << "\n";
std::cout << trapezoid(0.0, 1.0, 1000, fun) << "\n";
std::cout << trapezoid(0.0, 1.0, 10000000, fun) << "\n";
```
with results
```bash
0.3350000000000001
0.33333349999999995
0.33333333333333487
```
where fun is the square function.
:::


:::{exercise}
Now use your implementation to compute the error function as a function of $x$, using the partition shown. Your program must print also the relative error, comparing with the "exact" value from https://en.cppreference.com/w/cpp/numeric/math/erf . Print your data to `erf_data.txt` (print the x value, the numerical value of the integral, and the relative error)
\begin{equation}
f(x) = \frac{2}{\sqrt{\pi}}\int_0^x e^{-t^2} dt
\end{equation}

$x \in [0, 10]$, $\Delta x = 0.1$

$f(x=0), f(x=0+0.1), f(x=0+2\times 0.1), f(x=0+3\times 0.1), \ldots$
:::


You can use the following python script as a template script to plot the data

```python
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  
sns.set_context('poster')

x, erf, error = np.genfromtxt('erf_data.txt', unpack=True)
fig, ax = plt.subplots(1, 2)
ax[0].plot(x, erf, '-o', label="err function")
ax[0].legend()
ax[1].plot(x, error, '-s', label="relative error")
ax[1].legend()
ax[1].loglog()
```

## Simpsons rule
In this case, the method uses three discrete points and approximate the function as a parabola (second order polynomial), to improve the numerical integral computation. See: https://en.wikipedia.org/wiki/Simpson%27s_rule

|<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Simpsons_method_illustration.svg/440px-Simpsons_method_illustration.svg.png" width=300 /> |<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Simpsonsrule2.gif/440px-Simpsonsrule2.gif" />|
|-|-|
|2-order interpolation|Better results for more intervals|

Between two points, the integral is 
\begin{equation}
I(a, b, 2) = \int_a^b dx\ f(x) \simeq \frac{b-a}{6} \left[ f(a) + 4f(\frac{a+b}{2}) + f(b)\right] + O_{local}(\Delta x^5),
\end{equation}
so the weights are $\{1/6, 4/6, 1/6\}$.



Now, using a partition,  you compute the integral as
\begin{equation}
\int_a^b f(x) dx \simeq \frac{\Delta x}{3}\left( f(x_0) + 4 \sum_{k=1}^{N/2} f(x_{2k-1}) + 2\sum_{k=1}^{N/2-1} f(x_{2k}) + f(x_N)\right) + O_{global}(N^{-4}).
\end{equation}
Notice that $N$ must be even, and $\Delta x = \dfrac{b-a}{N}$, so $O_{global}(\Delta x^4) = O_{global}(N^{-4})$.

Now let's implement it. Please add a new function to the `integration.*` files , that implements the simpsons rule.

```c++
std::cout << simpson(0.0, 1.0, 10) << "\n";
```



Should give

```shell
0.3333333333333333
```

## Romberg method
The [Romberg Method](https://en.wikipedia.org/wiki/Romberg%27s_method) applies Richardson extrapolation to the given integration technique. The implementation is completely analogous as for the derivatives case.

:::{exercise}
Implement it for the previous method shown and compare the errors as a function of $N$, the number of intervals. Use the err function and fix $x = 0.54432$.
:::

### Other techniques
There are ways to improve the previous algorithms to decrease the error: using  higher order decompositions, using [Adaptative techniques](https://en.wikipedia.org/wiki/Adaptive_Simpson%27s_method), using [Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature), mixing some of these techniques, etc. But implementing those methods by hand would be a daunting task.

NOTE: When you have tabular data, you will be almost forced to use either the trapezoid or the simpsons method. You could also first interpolate the data and then integrate.

## Gaussian quadrature: Introduction to vectors
[Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature?useskin=vector) is a very clever technique that allows to compute integrals with a high precision and minimal number of points. First, notice that any numerical integral method can be written as

\begin{equation}
I(a, b) \simeq \sum w_k f(x_k) \Delta x,
\end{equation}
where the partition size can be $\Delta x = \dfrac{b-a}{N}$ and 
\begin{equation}
x_k = a + k \Delta x .
\end{equation}
Notice that the points coordinates are fixed. Here $x_k$ are the evaluation points and $w_k$ the associated weights. 

For example, going back to the previous techniques, the trapezoid rule can be written as ($N=1$) 
\begin{equation}
I (a, b) = \int_a^b dx \ f(x) \simeq \frac{b-a}{2} (f(a) + f(b)),
\end{equation}
so we have
- weights: $w = \{ 1/2, 1/2\}$
- points: $x = \{a, b\}$.

And for Simpsons rule we have (minimum two points)
\begin{equation}
I(a, b) = \int_a^b dx\ f(x) \simeq \frac{b-a}{6} \left[ f(a) + 4f(\frac{a+b}{2}) + f(b)\right],
\end{equation}
so we have 
- weights: $w = \{1/6, 4/6, 1/6\}$
- points: $\{a, (a+b)/2, b\}$.

What if you don't fix the $x_i$ to a uniform spacing, but, instead, try to compute both their optimal positions plus the optimal weights so you compute exactly the integral for an interpolating polynomial of degree $2n-1$, using $n$ points? that is the basic idea behind the Gaussian quadrature method. The modern formulation, thanks to Jacobi, uses orthogonal polynomials, usually in the [-1, 1] domain (**so you need to re-scale your data when you are integrating from $[a, b]$**). For instace using just to points will give an integral exact up to a polynomial of degree 3. **Just two points ...** 


<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/660px-Comparison_Gaussquad_trapezoidal.svg.png" alt="Image Description" width="600">
    <figcaption>From: "https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/660px-Comparison_Gaussquad_trapezoidal.svg.png"</figcaption>
</div>

In this image, it is illustrated that although the Gaussian method uses only two points, it is exact since the integration is done on a 3-degree polynomial. Here, the integrand is $y(x) = 7x^3 -8x^2 -rx -3$  . The exact integral value is 2/3. The trapezoid method, using two points, gives -10, forcing to use the composite trapezoid method, using much more numbers. The Gauss-Legendre, with just points (strategically chosen), gives the exact value.   

### Deduction of points and weights for the 2 point Gauss-Legendre method
The two-point rule approximates an integral as:
$$\int_{-1}^{1} f(x) \,dx \approx w_0 f(x_0) + w_1 f(x_1)$$
We need to find the four unknown parameters: the weights **$w_0, w_1$** and the points **$x_0, x_1$**. We can do this by requiring the formula to be perfectly accurate for polynomials of the highest possible degree, which for four unknowns is degree 3 ($2n-1 = 2(2)-1 = 3$).

---

#### Method 1: The Exactness Method

This method involves creating a system of equations by enforcing that the approximation is exact for $f(x) = 1, x, x^2,$ and $x^3$.

1.  **For $f(x) = 1$ (Degree 0):**
    * **Exact Integral:** $\int_{-1}^{1} 1 \,dx = 2$
    * **Approximation:** $w_0(1) + w_1(1) = w_0 + w_1$
    * **Equation 1:** $w_0 + w_1 = 2$

2.  **For $f(x) = x$ (Degree 1):**
    * **Exact Integral:** $\int_{-1}^{1} x \,dx = 0$
    * **Approximation:** $w_0(x_0) + w_1(x_1)$
    * **Equation 2:** $w_0 x_0 + w_1 x_1 = 0$

3.  **For $f(x) = x^2$ (Degree 2):**
    * **Exact Integral:** $\int_{-1}^{1} x^2 \,dx = \frac{2}{3}$
    * **Approximation:** $w_0(x_0^2) + w_1(x_1^2)$
    * **Equation 3:** $w_0 x_0^2 + w_1 x_1^2 = \frac{2}{3}$

4.  **For $f(x) = x^3$ (Degree 3):**
    * **Exact Integral:** $\int_{-1}^{1} x^3 \,dx = 0$
    * **Approximation:** $w_0(x_0^3) + w_1(x_1^3)$
    * **Equation 4:** $w_0 x_0^3 + w_1 x_1^3 = 0$

Now, we solve this system. Due to the symmetry of the interval $[-1, 1]$, we can assume a symmetric solution where **$w_0 = w_1$** and **$x_1 = -x_0$**.

* From Equation 1: If $w_0=w_1$, then $w_0 + w_0 = 2 \implies 2w_0 = 2 \implies \mathbf{w_0 = 1}$. Consequently, **$w_1 = 1$**.

* This solution ($w_0=w_1=1$ and $x_1=-x_0$) automatically satisfies Equations 2 and 4.

* Finally, we use Equation 3 to find the points:
    $$w_0 x_0^2 + w_1 x_1^2 = \frac{2}{3}$$
    $$(1) x_0^2 + (1) (-x_0)^2 = \frac{2}{3}$$
    $$x_0^2 + x_0^2 = \frac{2}{3}$$
    $$2x_0^2 = \frac{2}{3} \implies x_0^2 = \frac{1}{3}$$
    This gives the two points: **$x_0 = -\frac{1}{\sqrt{3}}$** and **$x_1 = \frac{1}{\sqrt{3}}$**.



#### Method 2: Using Legendre Polynomials

A more general method states that for an $n$-point rule, the points $x_i$ are the roots of the $n$-th degree Legendre Polynomial, $P_n(x)$.

For our two-point rule, $n=2$. The second-degree Legendre Polynomial is:
$$P_2(x) = \frac{1}{2}(3x^2 - 1)$$
Finding the roots means setting $P_2(x) = 0$:
$$\frac{1}{2}(3x^2 - 1) = 0 \implies 3x^2 = 1 \implies x = \pm \frac{1}{\sqrt{3}}$$
This confirms our points: **$x_0 = -1/\sqrt{3}$** and **$x_1 = 1/\sqrt{3}$**.

The corresponding weights $w_i$ are calculated with the formula:
$$w_i = \frac{2}{(1-x_i^2)[P'_n(x_i)]^2}$$
First, we need the derivative of $P_2(x)$:
$$P'_2(x) = \frac{d}{dx}\left[\frac{1}{2}(3x^2 - 1)\right] = 3x$$
Now we compute the weights:
* For **$x_0 = -1/\sqrt{3}$**:
    $w_0 = \frac{2}{(1-(-1/\sqrt{3})^2)[3(-1/\sqrt{3})]^2} = \frac{2}{(1-1/3)[-\sqrt{3}]^2} = \frac{2}{(2/3)(3)} = \mathbf{1}$

* For **$x_1 = 1/\sqrt{3}$**:
    $w_1 = \frac{2}{(1-(1/\sqrt{3})^2)[3(1/\sqrt{3})]^2} = \frac{2}{(1-1/3)[\sqrt{3}]^2} = \frac{2}{(2/3)(3)} = \mathbf{1}$



#### Summary

For the two-point Gauss-Legendre quadrature, the 0-indexed points and weights are:
* **Weights:** $w_0 = 1$, $w_1 = 1$
* **Points:** $x_0 = -\frac{1}{\sqrt{3}}$, $x_1 = \frac{1}{\sqrt{3}}$

The final approximation formula is:
$$\int_{-1}^{1} f(x) \,dx \approx (1)f\left(-\frac{1}{\sqrt{3}}\right) + (1)f\left(\frac{1}{\sqrt{3}}\right)$$

:::{exercise} Gauss-Legendre quadrature with 2 points
For the Gauss-Legendre formulation, you can check the weights at <https://en.wikipedia.org/wiki/Gaussian_quadrature?useskin=vector#Gauss%E2%80%93Legendre_quadrature>. Interpret the table. Now implement the 2 point Gauss-Legendre quadrature and test the figure results.
:::

:::{exercise}
Can Richardson extrapolation be applied to the Gauss quadrature method?
:::

## Scalling integration intervals

To successfully apply the Gaussian quadrature technique to a general integral on the interval [a, b], we need to map it to the interval [-1, 1]. Basically you want a new $x'$ such that $x = a$ ($b$) when $x' = 1$ (-1). Once such linear mapping is
\begin{equation}
x = \frac{b-a}{2}x' + \frac{b+a}{2},
\end{equation}
therefore
\begin{equation}
\int_a^b f(x) dx \simeq \frac{b-a}{2} \sum w_k f\left(\frac{b-a}{2}x_k' + \frac{b+a}{2}\right),
\end{equation}
using both the $w_i$ and $x_i'$ from the method.

:::{exercise} Gauss - Legendre on general intervals
Extend the previous to compute the integral on a general interval $[a, b]$. Test that it is still working for the original interval , and then compute the integral of $\sin(e^x)$ for $x \in [-5.0, 5.0]$, using the two point Gauss-Legendre method. Compare with both trapezoid and simpson with 2 points. How many points do you need in thee methods to achieve a comparable precision as gauss legendre 2 points? 


## Higher order Gauss-Legendre: using a vector to store the weights/points.
For an increasing number of points, you will need to use many variables to store the weights. For a degree $n=5$, you will need five variables to store the weights, five more for the points, and then manually process that. It will be much better to store them in an data structure that allow us to handle the operations in a generic way. In this case, it will be a `std::vector`, which allows us to store data, contiguously in memory, and process it using simple for loops (SIMD).

Our declaration could be
```cpp
double gauss_5(double a, double b, fptr fun);
```
and its implementation (you need to finish it)
```cpp
double gauss_5(double a, double b, fptr fun)
{
    std::vector<double> x(5); // store five points in the array
    std::vector<double> w(5); // store five weights in the array
    // initialize the data, sorted from left to right
    x[0] = -std::sqrt(5 + 2std::sqrt(10/7));
    x[1] = -std::sqrt(5 - 2std::sqrt(10/7));
    x[2] = 0.0;
    // ...
    w[0] = (322 - 13*std::sqrt(70))/(900);
    w[1] = (322 + 13*std::sqrt(70))/(900);
    w[2] = 128.0/255.0;
    //...
    // compute the integral
    double result = 0.0;
    for (int ii = 0; ii < 5; ++ii) {
        result += w[ii]*fun((b-a)*x[ii]/2 + (b+a)/2);
    }
    return result*???;
}
```

Notice how easy is to process data in an array.

Now finish the implementation, and print the error, as a function of $x$, computing the err function, comparing the relative errors  with trapezoid, simpson, trapezoid+richardson, simpson+trapezoid, gauss_5. Plot it.

## Exercises
### Gauss quadrature of order 7
(REF: Sirca, Computational Methods in Physics. Also
<https://en.wikipedia.org/wiki/Gaussian_quadrature>)

The next table shows the points positions and weights for the Gaussian integral
of order 7, $G_7$,

| Nodes position | Weigths |
|:---:|:---:|
| $\pm 0.949107912342759$ | 0.129484966168870 |
| $\pm 0.741531185599394$ | 0.279705391489277 |
| $\pm 0.405845151377397$ | 0.381830050505119 |
| $0.000000000000000$ | 0.417959183673469 |


1.  Implement the $G_7$ method. Integrate the function $\sin\sqrt x$ on the interval $[2,
       8.7]$. The exact indefinite integral is $2\sin\sqrt x - 2\sqrt x\cos\sqrt x$.
    
    <table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">
    
    
    <colgroup>
    <col  class="org-left" />
    
    <col  class="org-right" />
    </colgroup>
    <tbody>
    <tr>
    <td class="org-left">G7:</td>
    <td class="org-right">4.637955250321647</td>
    </tr>
    
    
    <tr>
    <td class="org-left">Exact:</td>
    <td class="org-right">4.637955200036554</td>
    </tr>
    </tbody>
    </table>

2.  Compute the err function as a function of $x \in [0, 10.0]$ in steps of
    $0.05$, using both $G_7$ and Simpson+Richardson and plot the relative
    difference with the &ldquo;exact&rdquo; function `std::erf`
    (<https://en.cppreference.com/w/cpp/numeric/math/erf>).

For more coefficients and higer orders, see <https://pomax.github.io/bezierinfo/legendre-gauss.html>


### Cumulative probability
The probability density function for the [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) is

\begin{equation}
f(x; \alpha, \beta) = \frac{x^{\alpha -1} \cdot e^{-x/\beta} }{\beta^\alpha \cdot \Gamma(\alpha)},
\end{equation}


para $0 < x < \infty$. Compute and plot the cumulative density function, using
both the simpson method and the gaussian $g_7$, defined as

\begin{equation}
F(x; \alpha, \beta) = \int_0^x dt f(t; \alpha, \beta),
\end{equation}

for $x \in [0.0, 20.0]$, with $\alpha = 7.5, \beta = 1.0$.



### Di-logarithm function
The di-logarithm function is defined as

\begin{equation}
\mathrm{Li}_2(x) = -\int_0^x \frac{\ln(1-t)}{t} dt.
\end{equation}

Plot the di-logarithm function for $x \in (0.0, 1.0)$. If you use simpson,
always make sure that the error is less than $10^{-6}$. You might need to
transform the variable and/or split the integral to avoid numerical problems.



### Moment distribution
The kth moment for a probability distribution is

\begin{equation}
\lt x^k \gt = \int x^k f(x) dx,
\end{equation}

Compute the first ten moments for the  [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) ,

\begin{equation}
f(x; \alpha, \beta) = \frac{\beta^\alpha}{\Gamma(\alpha) x^{\alpha -1} e^{-\beta x} }.
\end{equation}



### Arch lenght, mass, centroid

The arc length of a curve $y(x)$ is given by

\begin{equation}
s = \int_a^b dx \sqrt{1 + \left(\frac{dy}{dx}  \right)^2}.
\end{equation}

If a linear body has a density $\rho(x)$, the total mass would be

\begin{equation}
M = \int \rho(x) ds.
\end{equation}

The $x$ position of the centroid can also be computed as

\begin{equation}
\overline x = \frac{\int x\rho ds}{\int \rho ds}
\end{equation}

1.  (Boas, Mathematical Physics) Compute the arch length for the catenary $y =
       \cosh x$ between $x=-1$ and $x=1$.
2.  For a chain with shape $x^2$, with density $|x|$, between $x = -1$ and $x=1$,
    compute
    1.  The arc length
    2.  The total mass $M$
    3.  $\overline x$
    4.  $\overline y$



### Integrating a datafile

The following data (file `accel.txt`) represents the acceleration of a car as a
function of time

<table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">


<colgroup>
<col  class="org-right" />

<col  class="org-right" />
</colgroup>
<tbody>
<tr>
<td class="org-right">0.5</td>
<td class="org-right">2</td>
</tr>


<tr>
<td class="org-right">1</td>
<td class="org-right">3</td>
</tr>


<tr>
<td class="org-right">1.3</td>
<td class="org-right">3.6</td>
</tr>


<tr>
<td class="org-right">1.7</td>
<td class="org-right">4.4</td>
</tr>


<tr>
<td class="org-right">1.9</td>
<td class="org-right">4.15</td>
</tr>


<tr>
<td class="org-right">2.1</td>
<td class="org-right">3.85</td>
</tr>


<tr>
<td class="org-right">2.5</td>
<td class="org-right">3.25</td>
</tr>


<tr>
<td class="org-right">2.9</td>
<td class="org-right">2.65</td>
</tr>
</tbody>
</table>

Save the data into the corresponding file. Then write a program to read the data
and compute the velocity as a function of time by integrating at each time step.

